# Project Week 2

Emiel Steegh   - s1846388  
Freek Nijweide - s1857746

# Project Description

Description etc etc

In [ ]:
# The following includes are needed to work with graphs and display solutions.
%matplotlib inline
from __future__ import division

import numpy as np

import matplotlib.pyplot as plt
import utils
import graphs

import pandas as pd

plt.rcParams["figure.figsize"] = [12,8]

print("imports done")

## Assignment 1

text

## Assignment 2

text

## Assignment 3

In [1]:
# code

## Assignment 4

There are three different ranking systems discussed in this week's chapters and lectures.
The first is an inDegree-based ranking system. The ranking of a page depends on how many pages link to it.
This system is easily fooled, and dead-end pages receive incredibly high rankings.

The second ranking system discussed is Google's PageRank. This system expands upon the idea of inDegree rankings,
by having a node "spread" its incoming ranking over its outgoing links. This can also include a degree of randomness,
where a node will spread some of its incoming ranking over *all* nods (to avoid spider traps and dead ends).

The third ranking system discussed HITS, based on the hub-authority model. In this model, a page is considered a good hub
if it links to good authorities (pages with good content that people want to see). A page is considered a good
authority if it is linked to by proper hubs. The page's authority ranking is the one actually used in the search.


# write equations
# implementeer ze


In [2]:
# TODO 

# code
def generate_list_from_dict(dictionary):
    # TODO implement
    return 
    return new_list